<a href="https://colab.research.google.com/github/geopan2000/Exploring-Mental-Health-Data/blob/main/Mental-Health-Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Clone the GitHub repository
!git clone https://github.com/geopan2000/Exploring-Mental-Health-Data.git

# Change directory to the cloned repository
%cd Exploring-Mental-Health-Data

Cloning into 'Exploring-Mental-Health-Data'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 5.28 MiB | 3.00 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data


In [32]:
import pandas as pd

In [33]:
train_set = pd.read_csv('/content/Exploring-Mental-Health-Data/data/train.csv')
test_set = pd.read_csv('/content/Exploring-Mental-Health-Data/data/test.csv')

In [34]:
test_set.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
1,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
2,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
3,140703,Nalini,Female,23.0,Rajkot,Student,NaN,5.0,NaN,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
4,140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No


In [35]:
# Count rows where both columns are NaN
#both_nan_count = ((train_set['Job Satisfaction'].isna()) & (train_set['Study Satisfaction'].isna())).sum()

# Count rows where both columns have values
#both_filled_count = ((train_set['Job Satisfaction'].notna()) & (train_set['Study Satisfaction'].notna())).sum()

#print(f"Rows where both columns are NaN: {both_nan_count}")
#print(f"Rows where both columns have values: {both_filled_count}")

#print(train_set['Dietary Habits'].value_counts())

#print(train_set.isna().sum())
# .describe()
# train_set.info()

#for col in test_set.columns:
  #print(col, test_set[col].nunique())

In [36]:
train_set.drop(columns=['Name', 'id'], inplace=True)

train_set['Gender'] = train_set['Gender'].map({'Male' : 1, 'Female' : 0}) #Always trasnforming 2 option object types into binary

train_set['Age'] = train_set['Age'] / 100 #Normalized by the Max value
train_set['Age'] = train_set['Age'].fillna(train_set['Age'].mean())

train_set['City'] = train_set['City'].fillna('X')
train_set['City_freq'] = train_set['City'].map(train_set['City'].value_counts()) # frequency encoding, 98 different cities
train_set = pd.get_dummies(train_set, columns=['City'])

train_set['Working Professional or Student'] = train_set['Working Professional or Student'].map({'Working Professional' : 1, 'Student' : 0}) #Always trasnforming 2 option object types into binary

train_set['Profession'] = train_set['Profession'].fillna('X')
train_set['Profession_freq'] = train_set['Profession'].map(train_set['Profession'].value_counts()) # frequency encoding
train_set = pd.get_dummies(train_set, columns=['Profession'])

# Create the new feature 'Academic/Work Pressure'
train_set['Academic/Work Pressure'] = train_set.apply(
    lambda row: row['Academic Pressure'] if pd.notna(row['Academic Pressure']) and pd.isna(row['Work Pressure']) else
                row['Work Pressure'] if pd.isna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                (row['Academic Pressure'] + row['Work Pressure']) / 2 if pd.notna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in 'Academic/Work Pressure' with its mean
train_set['Academic/Work Pressure'] = train_set['Academic/Work Pressure'].fillna(train_set['Academic/Work Pressure'].mean())

train_set['Academic Pressure'] = train_set['Academic Pressure'] / 5
train_set['Academic Pressure'] = train_set['Academic Pressure'].fillna(0)

train_set['Work Pressure'] = train_set['Work Pressure'] / 5
train_set['Work Pressure'] = train_set['Work Pressure'].fillna(0)

train_set['CGPA'] = train_set['CGPA'] /10
train_set['CGPA'] = train_set['CGPA'].fillna(0)

# Create the new feature 'Study/Job Satisfaction'
train_set['Study/Job Satisfaction'] = train_set.apply(
    lambda row: row['Study Satisfaction'] if pd.notna(row['Study Satisfaction']) and pd.isna(row['Job Satisfaction']) else
                row['Job Satisfaction'] if pd.isna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                (row['Study Satisfaction'] + row['Job Satisfaction']) / 2 if pd.notna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in 'Study/Job Satisfaction' with its mean
train_set['Study/Job Satisfaction'] = train_set['Study/Job Satisfaction'].fillna(train_set['Study/Job Satisfaction'].mean())

train_set['Study Satisfaction'] = train_set['Study Satisfaction'] / 5
train_set['Study Satisfaction'] = train_set['Study Satisfaction'].fillna(0)

train_set['Job Satisfaction'] = train_set['Job Satisfaction'] / 5
train_set['Job Satisfaction'] = train_set['Job Satisfaction'].fillna(0)

train_set['Degree'] = train_set['Degree'].fillna('X')
train_set['Degree_freq'] = train_set['Degree'].map(train_set['Degree'].value_counts()) # frequency encoding
train_set = pd.get_dummies(train_set, columns=['Degree'])

train_set['Have you ever had suicidal thoughts ?'] = train_set['Have you ever had suicidal thoughts ?'].map({'Yes' : 1, 'No' : 0}) #Always trasnforming 2 option object types into binary

train_set['Work/Study Hours'] = train_set['Work/Study Hours'] / 24 #Normalized by the Max value

train_set['Financial Stress'] = train_set['Financial Stress'] / 5 #Normalized by the Max value

train_set['Family History of Mental Illness'] = train_set['Family History of Mental Illness'].map({'Yes' : 1, 'No' : 0}) #Always trasnforming 2 option object types into binary

In [37]:
 #Define a function to encode Sleep Duration
def encode_sleep_duration(value):
    if 'Less than 5 hours' in value or value in ['3-4 hours', '4-5 hours', '2-3 hours']:
        return 1  # < 5 hours
    elif '5-6 hours' in value:
        return 2  # 5-6 hours
    elif '6-7 hours' in value or '6-8 hours' in value:
        return 3  # 6-7 hours
    elif '7-8 hours' in value:
        return 4  # 7-8 hours
    elif 'More than 8 hours' in value or '8-9 hours' in value or '9-11 hours' in value:
        return 5  # > 8 hours
    else:
        return -1  # Placeholder for invalid or unclear values

# Apply the function to encode Sleep Duration
train_set['Sleep_Duration_Encoded'] = train_set['Sleep Duration'].apply(encode_sleep_duration)


In [29]:
# Define a function to extract From and To values
def parse_sleep_duration(value):
    if '-' in value:  # Ranges like '3-4 hours'
        parts = value.split('-')
        return int(parts[0]), int(parts[1].split()[0])
    elif '>' in value:  # Values like '>7 hours'
        lower_bound = int(value.split('>')[1].split()[0])
        return lower_bound, 12  # Assume upper bound is 12
    elif '<' in value or 'Less than' in value:  # Values like '<5 hours' or 'Less than 5 hours'
        upper_bound = int(value.split()[-2]) if 'Less than' in value else int(value.split('<')[1].split()[0])
        return 0, upper_bound
    elif 'More than' in value:  # Values like 'More than 8 hours'
        lower_bound = int(value.split()[-2])
        return lower_bound, 12  # Assume upper bound is 12
    else:
        return None, None  # For invalid or unclear values

# Apply the function to create 'From' and 'To' columns
train_set[['From', 'To']] = train_set['Sleep Duration'].apply(lambda x: pd.Series(parse_sleep_duration(x)))

In [30]:
# Define mapping for valid dietary habits
dietary_mapping = {
    'Moderate': 'Moderate',
    'Unhealthy': 'Unhealthy',
    'Healthy': 'Healthy',
    'More Healthy': 'Healthy',
    'Less Healthy': 'Unhealthy',
    'Less than Healthy': 'Unhealthy',
    # Ambiguous or irrelevant entries
    'Yes': None, 'No': None, 'No Healthy': None, 'Class 12': None, 'Indoor': None,
    'Male': None, 'Vegas': None, 'M.Tech': None, '1.0': None, 'Electrician': None,
    'Hormonal': None, 'Mihir': None, '3': None, 'Gender': None, 'BSc': None,
    'Pratham': None, '2': None
}

# Apply the mapping to the Dietary Habits column
train_set['Dietary Habits'] = train_set['Dietary Habits'].map(dietary_mapping)
train_set['Dietary Habits'] = train_set['Dietary Habits'].fillna('X')
train_set['Dietary Habits_freq'] = train_set['Dietary Habits'].map(train_set['Dietary Habits'].value_counts()) # frequency encoding
train_set = pd.get_dummies(train_set, columns=['Dietary Habits']) # dummy variables

In [41]:
train_set.shape
train_set.to_csv('train_set.csv', index=False)